# Introduction to models

With normal programming, we have:

Rules + Input   -> Output

With machine learning, we have:

Input + Expected Output -> Rules.

This notebook runs through a very simple machine learning algorithm: Linear Regression.

## My First Model: Linear Regression

### The problem

In the file `cluster.csv`, there are measurements of how many emails were processed in an hour by a cluster of `n` cores. We want to find out how much capacity we gain per core added.

In other words, we want to find `w` such that:

`emails_processed = w * number_of_cores`.

This is an example of a _regression problem_.

Given `X` number of cores, predict `y`, the number of emails processed.

### Setting up

We first use the `pandas` package to load our data. `pandas`, and `numpy` are essential packages for data management in machine learning with python. You can see some cheat sheets for these packages here:

[pandas](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)

[numpy](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Numpy_Python_Cheat_Sheet.pdf)

### Training

### Interpreting the model

### Evaluating the model

# Multiple Linear Regression
